In [14]:
import stealth_requests as requests
import requests as r
from bs4 import BeautifulSoup
from markdownify import markdownify as md
from dotenv import load_dotenv
import os
import json
load_dotenv()

True

In [15]:
urls = [
    "https://summer.skku.edu/summer/intro/welcome.do", # Welcome to SKKU Summer
    "https://summer.skku.edu/summer/intro/why.do", # Why Summer at SKKU?
    "https://summer.skku.edu/summer/program/overview.do", # Overview of SKKU Summer
    "https://summer.skku.edu/summer/program/2025Exclusive.do", #2025 Exclusive : Jeju Cultural Journey
    "https://summer.skku.edu/summer/program/activities.do", # Cultural Activity
    "https://summer.skku.edu/summer/life/Student_Service.do", # Student Services
    "https://summer.skku.edu/summer/program/policies.do", # Academic Policies
    ]

In [16]:
def scrape_content(url):
    resp = requests.get(url)
    soup = BeautifulSoup(resp.content, 'html.parser')

    # Find the 'div' with class 'content-wrap'
    content_wrap_div = soup.find('div', class_='content-wrap')

    # Get the HTML content of this div
    if content_wrap_div:
        content_wrap_html = str(content_wrap_div)
        md_content = md(content_wrap_html)
        print(md_content)
        return md_content
    else:
        print("Div with class 'content-wrap' not found.")
        return None


In [17]:
import base64

markdown_dicts = []
for url in urls:
    print(f"Scraping {url}...")
    url = url
    content = scrape_content(url)
    markdown_bytes = content.encode('utf-8')
    base64_bytes = base64.b64encode(markdown_bytes)
    if content:
        markdown_dicts.append({
            "url": url,
            "content": content,
            "base64": base64_bytes.decode('utf-8')
        })


Scraping https://summer.skku.edu/summer/intro/welcome.do...
* HOME
* Why SKKU
* Welcome

### Welcome

#### Welcome to Sungkyunkwan University's (SKKU) International Summer Semester (ISS) !

As one of Asia’s oldest yet most cutting-edge research universities, SKKU connects the past, present, and future through a unique balance of tradition and innovation. In the pursuit of becoming a world-renowned university, the ISS has become an important channel of global academic exchange since its establishment in 2008.

By offering the most comprehensive and innovative program in Korea, the ISS has attracted thousands of students interested in looking beyond borders. The ISS has become one of the most successful and diverse summer programs in Asia, receiving 1,434 students from 40 different countries (104 institutions) in 2019.

I am certain that the ISS is a truly unique and fun summer experience. The ISS provides not only excellent academic programs, but also helps students take a journey of pe

In [ ]:
api_url = "http://localhost:8080/"
api_key = os.getenv("OPEN_WEBUI_API_KEY")
headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json",
    'Accept': 'application/json'
}

In [19]:
knowledge_base_data = {
    "name": "ISS Info",
    "description": "Knowledge Base with information about International Summer Semester",
    "data": {},
    "access_control": {}
}

# The api_url is defined in cell 4
create_kb_url = f"{api_url}api/v1/knowledge/create"
response = r.post(create_kb_url, headers=headers, json=knowledge_base_data)
if response.status_code == 200:
    kb_creation_response = response.json()
    kb_creation_id = kb_creation_response.get("id")
    print(kb_creation_response)
else:
    print(f"Failed to create knowledge base: {response.status_code} - {response.text}")

{'id': '882dfcbb-b5e4-4b30-b1dc-8a23d5ae27ec', 'user_id': '34ee361c-d645-4614-82e4-53b2176e8e2c', 'name': 'ISS Info', 'description': 'Knowledge Base with information about International Summer Semester', 'data': {}, 'meta': None, 'access_control': {}, 'created_at': 1748489294, 'updated_at': 1748489294, 'files': None}


In [22]:
kb_creation_id

'882dfcbb-b5e4-4b30-b1dc-8a23d5ae27ec'

In [23]:
api_add_file_url = "/api/v1/files/"

In [24]:
def add_file_to_kb(source_url, base64_content):
    api_add_file_url = "/api/v1/files/"
    files = {'file': base64_content}
    response = r.post(f"{api_url}{api_add_file_url}", headers=headers, files=files)
    return response

In [ ]:
add_file_to_kb